# Benders' decomposition example

$$
\begin{align*}
\min \quad & c'x + f'y \\
\mathrm{s.t.} \quad & \sum_i x_{ij} \geq d_j \\
& -\sum_j x_{ij} \geq -s_i \\
& -x_{ij} \geq -x^U_{ij} \cdot y_{ij}\\
& x\geq 0 \\
& y \in \{0,1\}
\end{align*}
$$

### Master
$$
\begin{align*}
\min_{y,z} \quad & z \\
\mathrm{s.t.}  \quad & z \geq  f'y + w^d d - w^s s - w^u x^U y, \quad \forall  k \in \mathrm{OptimalityCuts} \\
& w^d d - w^s s - w^u x^U y \leq 0, \quad \forall k \in \mathrm{FeasibilityCuts} \\
& y \in \{0,1\} \\
& z \geq 0
\end{align*}
$$

### Sub-problem
$$
\begin{align*}
\min_x \quad & z = f'y^* + c'x \\
\mathrm{s.t.} \quad & \sum_i x_{ij} \geq d_j \\
& -\sum_j x_{ij} \geq -s_i \\
& -x_{ij} \geq -x^U_{ij} y^*_{ij} \\
& x\geq 0
\end{align*}
$$

### Dual of sub-problem
$$
\begin{align*}
\max \quad & f'y^* + d w^d - s w^s - x^U y^* w^u \\
\mathrm{s.t.} \quad & w^d - w^s - w^u \leq c \\
& w^d, w^s, w^u \geq 0
\end{align*}
$$

In [411]:
from gurobipy import *

In [412]:
import numpy as np

supply = [10.,30.,40.,20]
demand = [20.,50.,30.]
varcost = [[2.,3.,4.], [3.,2.,1.], [1.,4.,3.], [4.,5.,2.]]
fixedcost = [[10.,30.,20.], [10.,30.,20.], [10.,30.,20.], [10.,30.,20.]]
xup = {(i,j):min(supply[i], demand[j]) for i in range(ns) for j in range(nd)}

In [413]:
UB = 1e3

ns = len(supply)
nd = len(demand)

### Master problem
master = Model('master')
master.Params.LazyConstraints = 1

ydict = {(i,j):master.addVar(0,1,0., GRB.BINARY, 'y[%d,%d]'%(i,j)) for i in range(ns) for j in range(nd)}
z = master.addVar(0, UB, 0., GRB.CONTINUOUS, 'z')
# Turn off presolve for master
master.Params.Presolve = 0

master.update()
f = fixedcost
master.addConstr(z >= sum([f[i][j]*ydict[(i,j)] for i in range(ns) for j in range(nd)]))
master.setObjective(z, GRB.MINIMIZE)
#master.addConstr(z >= sum([f[i][j]*ydict[(i,j)] for i in range(ns) for j in range(nd)]))
#master.setObjective(z + sum([f[i][j]*ydict[(i,j)] for i in range(ns) for j in range(nd)]), GRB.MINIMIZE)
master.update()

Changed value of parameter LazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1


In [414]:
### Subproblem
sub = Model('sub')
sub.Params.OutputFlag=0
ws = {i:sub.addVar(0, UB, 0., GRB.CONTINUOUS, 'ws[%d]'%i) for i in range(ns)}
wd = {i:sub.addVar(0, UB, 0., GRB.CONTINUOUS, 'wd[%d]'%i) for i in range(nd)}
wu = {(i,j):sub.addVar(0, UB, 0., GRB.CONTINUOUS, 'wu[%d]'%i) for i in range(ns) for j in range(nd)}
sub.update()
# Static subproblem constraint
c = varcost
cons_dual = {(i,j):sub.addConstr(wd[j]-ws[i]-wu[(i,j)] <= c[i][j]) for i in range(ns) for j in range(nd)}
sub.update()

In [415]:
GAPTOL = 1e-6

gap_best = 1e6

### Feas and Opt cuts implemented as lazy constraints
def benders(model, where):
    if where in (GRB.Callback.MIPSOL, GRB.Callback.MIPNODE):
        ### Lazy constraints only allowed for MIPNODE or MIPSOL
        ### Update subproblem objective
        print('*'*40)
        print('In CALLBACK where: %s' % where)
        print('*'*40)
        if where==GRB.Callback.MIPSOL:
            yopt = {(i,j):model.cbGetSolution(ydict[(i,j)]) for i in range(ns) for j in range(nd)}
            zmaster = model.cbGetSolution(z)
        elif where==GRB.Callback.MIPNODE:        
            yopt = {(i,j):model.cbGetNodeRel(ydict[(i,j)]) for i in range(ns) for j in range(nd)}            
            zmaster = model.cbGetNodeRel(z)
        
        subobj = sum([demand[j]*wd[j] for j in range(nd)]) - \
                     sum([supply[i]*ws[i] for i in range(ns)]) - \
                     sum([xup[(i,j)]*yopt[(i,j)]*wu[(i,j)] for i in range(ns) for j in range(nd)])
        sub.setObjective(subobj, GRB.MAXIMIZE)
#         print('*'*40)
#         print('Set subproblem objective')
#         print('*'*40)
        ### Solve updated subproblem
        sub.update()
        sub.optimize()
        print('*'*40)
        print('Subproblem solved with status: %s' % sub.Status)
        print('*'*40)
        if sub.Status == GRB.Status.UNBOUNDED:
            # Add feasibility cut, ensuring that cut indeed eliminates current incumbent
            print('*'*40)
            print('Adding feasibility cut')
            print('*'*40)
            # Get unbounded ray
            rays = [ws[i].X for i in range(ns)]
            rayd = [wd[i].X for i in range(nd)]
            rayu = {(i,j):wu[(i,j)].X for i in range(ns) for j in range(nd)}
            feascut = sum([demand[j]*rayd[j] for j in range(nd)]) - \
                sum([supply[i]*rays[i] for i in range(ns)]) - \
                sum([xup[(i,j)]*rayu[(i,j)]*y[(i,j)] for i in range(ns) for j in range(nd)]) <= 0
            model.cbLazy(feascut)
            ### Was the current incumbent eliminated?
        else:
            ### Otherwise add Optimality cut            
            try:
                zsub = sub.ObjVal + sum([f[i][j]*yopt[(i,j)] for i in range(ns) for j in range(nd)])
            except:
                zsub = -1e6
            gap = zmaster - zsub
            #***
            print('*'*40)    
            print('zSub: %g' % zsub)
            print('zMaster: %g' % zmaster)                        
            print('Gap: %g' % gap)
            print('*'*40)
            #***
            ### Check for termination
            if abs(gap) > GAPTOL:                
                # Need to add optimality cut since master
                # overestimated (for maximization) actual obj
                wsopt = [ws[i].X for i in range(ns)]
                wdopt = [wd[i].X for i in range(nd)]
                wuopt = {(i,j):wu[(i,j)].X for i in range(ns) for j in range(nd)}                
                optcut = z >= sum([f[i][j]*ydict[(i,j)] for i in range(ns) for j in range(nd)]) + \
                    sum([demand[j]*wdopt[j] for j in range(nd)]) - \
                    sum([supply[i]*wsopt[i] for i in range(ns)]) - \
                    sum([xup[(i,j)]*wuopt[(i,j)]*ydict[(i,j)] for i in range(ns) for j in range(nd)])
                model.cbLazy(optcut)
                print('*'*40)
                print('Added optimality cut')
                print('*'*40)
            else:                
                # Accept as new incumbent                                
                #if gap < gap_best:
                #    gap_best = gap
                model._x_best = {k:v.Pi for k,v in cons_dual.items()}
#                 print('!'*40)
#                 print('Accepting as new incumbent: z = %g' % zmaster)
#                 print('!'*40)

In [416]:
master.Params.OutputFlag=0
master._x_best = {}
master.optimize(benders)

****************************************
In CALLBACK where: 4
****************************************
****************************************
Subproblem solved with status: 2
****************************************
****************************************
zSub: 100000
zMaster: 0
Gap: -100000
****************************************
****************************************
Added optimality cut
****************************************
****************************************
In CALLBACK where: 4
****************************************
****************************************
Subproblem solved with status: 2
****************************************
****************************************
zSub: 100000
zMaster: 0
Gap: -100000
****************************************
****************************************
Added optimality cut
****************************************
****************************************
In CALLBACK where: 5
****************************************
*****************

In [417]:
master.ObjVal

350.0000000000057

In [418]:
sub.ObjVal + sum([f[i][j]*ydict[(i,j)].X for i in range(ns) for j in range(nd)])

100110.0

In [419]:
sub.ObjVal

100000.0

In [391]:
### Benchmark against non-Benders'

In [392]:
milp = Model('milp')
x0 = {(i,j):milp.addVar(0, UB, 0., GRB.CONTINUOUS, 'x[%d,%d]' % (i,j)) for i in range(ns) for j in range(nd)}
y0 = {(i,j):milp.addVar(0, 1, 0., GRB.BINARY, 'y[%d,%d]'%(i,j)) for i in range(ns) for j in range(nd)}
milp.update()
cons_supply = [milp.addConstr( sum([x0[(i,j)] for j in range(nd)]) <= supply[i])  for i in range(ns)]
cons_demand = [milp.addConstr( sum([x0[(i,j)] for i in range(ns)]) >= demand[j]) for j in range(nd)]
cons_bigM   = [milp.addConstr(  x0[(i,j)] <= xup[(i,j)]*y0[(i,j)]) for i in range(ns) for j in range(nd)]
f = fixedcost
c = varcost
milp.setObjective(sum([f[i][j]*y0[(i,j)] + c[i][j]*x0[(i,j)] for i in range(ns) for j in range(nd)]),
                  GRB.MINIMIZE)

In [393]:
milp.optimize()

Optimize a model with 19 rows, 24 columns and 48 nonzeros
Variable types: 12 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [1e+00, 3e+01]
  Bounds range     [1e+00, 1e+03]
  RHS range        [1e+01, 5e+01]
Found heuristic solution: objective 390
Presolve time: 0.00s
Presolved: 19 rows, 24 columns, 48 nonzeros
Variable types: 12 continuous, 12 integer (12 binary)

Root relaxation: objective 3.216667e+02, 16 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  321.66667    0    3  390.00000  321.66667  17.5%     -    0s
H    0     0                     360.0000000  321.66667  10.6%     -    0s
H    0     0                     350.0000000  335.00000  4.29%     -    0s
     0     0  348.33333    0    3  350.00000  348.33333  0.48%     -    0s

Cutting planes:
  Gomory: 1
  Cover: 2
  MIR: 

In [420]:
milp.ObjVal

350.0

In [421]:
master.ObjVal

350.0000000000057

In [422]:
print('%-15.10s%-15.10s%-15.10s' % ('Original','Benders', 'diff'))
for i in range(ns):
    for j in range(nd):
        print('%-15.10g%-15.3g%-15.3g' % (y0[(i,j)].X, ydict[(i,j)].X, ydict[(i,j)].X-y0[(i,j)].X))

Original       Benders        diff           
0              -0             -0             
-0             -0             0              
1              1              0              
-0             2.8e-13        2.8e-13        
1              1              0              
-0             -0             0              
1              1              -2.8e-13       
1              1              0              
0              -0             -0             
-0             -0             0              
-0             -0             0              
1              1              0              


In [397]:
#xdict = {k:v.Pi for k,v in cons_dual.items()}
#xdict

In [423]:
x_best = master._x_best

In [424]:
print('%-15.10s%-15.10s%-15.10s' % ('Original','Benders', 'diff'))
for i in range(ns):
    for j in range(nd):
        print('%-15.10g%-15.3g%-15.3g' % (x0[(i,j)].X, x_best[(i,j)], x_best[(i,j)]-x0[(i,j)].X))

Original       Benders        diff           
0              10             10             
0              0              0              
10             0              -10            
0              0              0              
30             30             0              
0              0              0              
20             10             -10            
20             20             0              
0              10             10             
0              0              0              
0              0              0              
20             20             0              


In [425]:
sub.ObjVal

100000.0

### Alternate optima exist

In [432]:
sum([c[i][j]*x_best[(i,j)] for i in range(ns) for j in range(nd)])

240.0

In [433]:
sum([c[i][j]*x0[(i,j)].X for i in range(ns) for j in range(nd)])

240.0

In [427]:
sub.ObjVal + sum([f[i][j]*ydict[(i,j)].X for i in range(ns) for j in range(nd)])

100110.0

In [428]:
sum([c[i][j]*x_best[(i,j)] for i in range(ns) for j in range(nd)]) + sum([f[i][j]*ydict[(i,j)].X for i in range(ns) for j in range(nd)])

350.0

In [429]:
master.ObjVal

350.0000000000057

### Restricted Master Problem
$$
\begin{align*}
\min_{y,z} \quad & z \\
\mathrm{s.t.} \quad & z \geq f'y + (d-By)'w^A_{i,k} + l_k'w^l_{i,k} - u'_k w^u_{i,k}, & i \in \mathrm{OptimalityCuts}, \ k=1,\dots,K \\
& (d_k-By)'w^A_{i,k} + l_k' w^l_{i,k} - u_k' w^u_{i,k} \leq 0, & i \in \mathrm{FeasibilityCuts}, \ k=1,\dots,K
\end{align*}
$$